# Building AI Agents with CrewAI using Python

## 🎯 Learning Objectives

By the end of this notebook, you will:
- ✅ Understand the fundamentals of CrewAI and multi-agent orchestration
- ✅ Learn how to define agents with roles, goals, and backstories
- ✅ Master task creation and dependency management
- ✅ Build a complete market strategy crew with multiple agents
- ✅ Understand LLM integration (both local and cloud-based)
- ✅ Learn advanced patterns: delegation, memory, and tool usage
- ✅ Build complex multi-agent systems for real-world scenarios
- ✅ Practice with hands-on exercises and challenges

## 📚 What You'll Build

1. **Basic Market Strategy Crew**: A two-agent system for competitive analysis
2. **Research & Writing Crew**: A three-agent system for content creation
3. **Code Review Crew**: A multi-agent system for software quality assurance
4. **Product Development Crew**: A complex system with multiple specialized agents

## 🧠 Key Concepts You'll Learn

1. **Agent Definition**: Creating specialized AI agents with unique roles
2. **Task Orchestration**: Coordinating multiple tasks with dependencies
3. **Crew Management**: Bringing agents together to solve complex problems
4. **LLM Integration**: Using both local and cloud-based language models
5. **Agent Memory**: Enabling agents to remember context across tasks
6. **Tool Integration**: Giving agents access to external tools and APIs
7. **Delegation**: Allowing agents to collaborate and delegate work

---

## ⚠️ Prerequisites

Before starting, make sure you have:
1. Python 3.8+ installed
2. Required packages installed (see next cell)
3. (Optional) API keys for cloud LLMs (OpenAI, Anthropic, etc.)
4. (Optional) Local LLM model file if using local inference

**Note**: This notebook demonstrates both local (offline) and cloud-based LLM options.


## 📦 Step 1: Install Required Packages

CrewAI requires several packages. Let's install them:

**Core Packages:**
- `crewai`: The main framework for building multi-agent systems
- `langchain`: Required dependency for CrewAI
- `python-dotenv`: For managing environment variables (API keys)

**Optional Packages (for local LLMs):**
- `llama-cpp-python`: For running local GGUF models
- `transformers`: For Hugging Face models
- `accelerate`: For model optimization

**Note**: If you're using cloud-based LLMs (OpenAI, Anthropic, etc.), you may need additional packages like `openai` or `anthropic`.


## 🔧 Troubleshooting

### Widget Rendering Error Fix

If you encounter the error: **"Error rendering output item using 'jupyter-ipywidget-renderer' i is not a function"**, this is caused by corrupted widget outputs in the notebook.

**Quick Fix:** Run the cell below to clean all widget outputs from the notebook.

**Prevention:** This usually happens when:
- Widget outputs get corrupted during notebook save/load
- ipywidgets version incompatibilities
- Jupyter server restarts while widgets are active

**Note:** The notebook has been pre-cleaned, but if the error appears again, use the utility below.

In [ ]:
# Utility: Clean corrupted widget outputs from notebook
# Run this cell if you get widget rendering errors

import json
import os

def clean_widget_outputs(notebook_path):
    """Remove all widget outputs from a Jupyter notebook."""
    with open(notebook_path, 'r') as f:
        nb = json.load(f)
    
    cleaned_count = 0
    for cell in nb['cells']:
        if cell['cell_type'] == 'code':
            outputs = cell.get('outputs', [])
            # Filter out widget outputs
            cleaned_outputs = []
            for output in outputs:
                if 'data' in output:
                    if 'application/vnd.jupyter.widget-view+json' not in output.get('data', {}):
                        cleaned_outputs.append(output)
                    else:
                        cleaned_count += 1
                else:
                    cleaned_outputs.append(output)
            cell['outputs'] = cleaned_outputs
    
    # Save cleaned notebook
    with open(notebook_path, 'w') as f:
        json.dump(nb, f, indent=1)
    
    return cleaned_count

# Get current notebook path
notebook_path = os.path.abspath('07 - Building AI Agents with CrewAI using Python.ipynb')
if not os.path.exists(notebook_path):
    notebook_path = os.path.abspath('./src/07 - Building AI Agents with CrewAI using Python.ipynb')

if os.path.exists(notebook_path):
    count = clean_widget_outputs(notebook_path)
    if count > 0:
        print(f"✅ Cleaned {count} widget outputs from notebook")
        print("   Please reload the notebook to see the changes.")
    else:
        print("✅ No widget outputs found - notebook is clean!")
else:
    print(f"⚠️ Notebook not found at: {notebook_path}")
    print("   Please update the path in this cell.")

In [1]:
# Install required packages
# Uncomment the line below if you need to install packages
# %pip install crewai langchain python-dotenv -q

# For local LLM support (optional):
# %pip install llama-cpp-python transformers accelerate -q

# Verify installation
try:
    import crewai
    print("✅ CrewAI installed successfully!")
    print(f"📦 CrewAI version: {crewai.__version__ if hasattr(crewai, '__version__') else 'N/A'}")
except ImportError as e:
    print(f"❌ Error importing CrewAI: {e}")
    print("Please run: pip install crewai langchain python-dotenv")


✅ CrewAI installed successfully!
📦 CrewAI version: 1.7.2


## 🔧 Step 2: Understanding CrewAI Architecture

Before we start coding, let's understand the key components of CrewAI:

### Core Components

1. **Agent**: An AI entity with:
   - **Role**: What the agent does (e.g., "Market Research Analyst")
   - **Goal**: What the agent aims to achieve
   - **Backstory**: Context that shapes the agent's behavior
   - **LLM**: The language model powering the agent
   - **Tools**: External capabilities the agent can use
   - **Memory**: Ability to remember past interactions

2. **Task**: A unit of work that:
   - Has a description of what needs to be done
   - Is assigned to a specific agent
   - Can depend on other tasks
   - Has an expected output format

3. **Crew**: The orchestrator that:
   - Manages multiple agents
   - Coordinates task execution
   - Handles dependencies between tasks
   - Returns the final result

### Workflow Pattern

```
Agent 1 → Task 1 → Output 1
                ↓
Agent 2 → Task 2 (depends on Task 1) → Output 2
                ↓
Agent 3 → Task 3 (depends on Task 2) → Final Output
```

This pattern allows agents to build upon each other's work, creating a collaborative system.


## 🔑 Step 3: LLM Configuration

CrewAI supports multiple LLM providers. This notebook demonstrates both local and cloud-based options.

### 🎛️ LLM Provider Selection

**Choose your LLM provider:**
- **Local (Ollama)**: Free, private, runs on your machine
- **Cloud (Gemini)**: Free tier available, better performance, requires API key

You can easily switch between them using the `LLM_PROVIDER` variable below.

### Available Options

1. **Ollama (Local)**: 
   - ✅ Free and private
   - ✅ Works offline
   - ⚠️ Requires Ollama installed and running
   - ⚠️ Slower than cloud options

2. **Google Gemini (Cloud)**:
   - ✅ Free tier available
   - ✅ Better performance
   - ✅ Easy setup with API key
   - ⚠️ Requires internet connection
   - ⚠️ Requires GEMINI_API_KEY in .env file


In [2]:
# ============================================================================
# 🎛️ LLM PROVIDER SELECTION
# ============================================================================
# Choose your LLM provider: "ollama" or "gemini"
# Change this variable to switch between providers
LLM_PROVIDER = "gemini"  # Options: "ollama" or "gemini"

print("="*70)
print(f"🔧 Selected LLM Provider: {LLM_PROVIDER.upper()}")
print("="*70)

# ============================================================================
# Option 1: Using OpenAI (Cloud-based) - Optional
# ============================================================================
# Uncomment and configure if you have an OpenAI API key

# import os
# from dotenv import load_dotenv
# from langchain_openai import ChatOpenAI

# load_dotenv()
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# ============================================================================
# Option 2a: Using Google Gemini (Cloud-based, FREE tier available)
# ============================================================================
if LLM_PROVIDER.lower() == "gemini":
    # Requires: GEMINI_API_KEY in .env file
    # Get your free API key from: https://aistudio.google.com/apikey
    
    try:
        from crewai import LLM as CrewAILLM
        from dotenv import load_dotenv
        import os
        
        # Load environment variables from .env file
        # Try to find .env file in current directory or parent directory
        env_paths = ['.env', '../.env', '../../.env']
        for env_path in env_paths:
            if os.path.exists(env_path):
                load_dotenv(env_path)
                break
        else:
            # If .env not found, try default behavior
            try:
                load_dotenv()
            except:
                pass  # Environment variables might already be set
        
        # Check for API key (CrewAI accepts both GEMINI_API_KEY and GOOGLE_API_KEY)
        if 'GEMINI_API_KEY' in os.environ or 'GOOGLE_API_KEY' in os.environ:
            # CrewAI supports Gemini with format "gemini/model-name"
            # Available free models: gemini-2.5-flash (fast), gemini-2.5-pro (better quality)
            # Note: CrewAI strips the "gemini/" prefix, so use: gemini-2.5-flash, gemini-2.5-pro, etc.
            gemini_model = "gemini/gemini-2.5-flash"  # Fast and free - recommended
            # Alternative: "gemini/gemini-2.5-pro" for better quality (slower)
            # Other options: "gemini/gemini-flash-latest", "gemini/gemini-pro-latest"
            
            try:
                llm_wrapper = CrewAILLM(
                    model=gemini_model,
                    temperature=0.7,
                    # CrewAI will automatically use GEMINI_API_KEY or GOOGLE_API_KEY from environment
                )
                USE_LOCAL_LLM = True  # Flag indicates "LLM is configured"
                print(f"✅ Gemini LLM configured successfully!")
                print(f"   Model: {gemini_model}")
                api_key_source = "GEMINI_API_KEY" if 'GEMINI_API_KEY' in os.environ else "GOOGLE_API_KEY"
                print(f"   API Key: Found ({api_key_source})")
                print(f"   Provider: Google Gemini (Cloud)")
            except Exception as e:
                print(f"❌ Error creating Gemini LLM: {e}")
                print("   Make sure GEMINI_API_KEY is set in .env file")
                llm_wrapper = None
                USE_LOCAL_LLM = False
        else:
            print("❌ GEMINI_API_KEY not found in environment")
            print("   1. Add GEMINI_API_KEY to your .env file")
            print("   2. Get your free API key from: https://aistudio.google.com/apikey")
            print("   3. Restart the notebook kernel after adding the key")
            llm_wrapper = None
            USE_LOCAL_LLM = False
    except ImportError as e:
        print(f"❌ Error importing CrewAI: {e}")
        llm_wrapper = None
        USE_LOCAL_LLM = False

# ============================================================================
# Option 2b: Using Ollama (Local LLM)
# ============================================================================
elif LLM_PROVIDER.lower() == "ollama":
    # Ollama is the easiest way to use local LLMs with CrewAI
    # Steps:
    # 1. Install Ollama: https://ollama.ai
    # 2. Pull a model: ollama pull mistral:7b (or ollama pull mistral)
    # 3. Start Ollama: ollama serve (or it may run automatically)
    # 4. Use CrewAI's LLM class with format "ollama/model_name:tag"
    
    try:
        from crewai import LLM as CrewAILLM
        import subprocess
        import requests
        
        # Check if Ollama is running and get available models
        try:
            result = subprocess.run(['ollama', 'list'], capture_output=True, text=True, timeout=2)
            if result.returncode == 0:
                print("✅ Ollama is installed and running")
                
                # Try to get models from Ollama API
                available_models = []
                try:
                    response = requests.get('http://localhost:11434/api/tags', timeout=2)
                    if response.status_code == 200:
                        models_data = response.json()
                        available_models = [model['name'] for model in models_data.get('models', [])]
                        if available_models:
                            print(f"   Available models: {', '.join(available_models[:5])}")
                except:
                    pass
                
                # Try to find a suitable model (prefer mistral variants)
                model_to_use = None
                preferred_models = ['mistral:7b', 'mistral', 'llama3.1:8b', 'llama3.1', 'llama3:latest']
                
                for pref in preferred_models:
                    if pref in available_models:
                        model_to_use = pref
                        break
                
                # If no preferred model found, use the first available
                if not model_to_use and available_models:
                    model_to_use = available_models[0]
                
                if model_to_use:
                    # IMPORTANT: Use format "ollama/model_name:tag" (include the tag if present)
                    ollama_model_name = f"ollama/{model_to_use}"
                    try:
                        llm_wrapper = CrewAILLM(model=ollama_model_name, temperature=0.7)
                        USE_LOCAL_LLM = True
                        print(f"✅ Using Ollama with '{model_to_use}' model via CrewAI")
                        print(f"   Format: {ollama_model_name}")
                        print(f"   Provider: Ollama (Local)")
                    except Exception as e:
                        print(f"⚠️ Error creating CrewAI LLM with Ollama: {e}")
                        print(f"   Tried model: {ollama_model_name}")
                        print("   Make sure the model is pulled: ollama pull mistral:7b")
                        llm_wrapper = None
                        USE_LOCAL_LLM = False
                else:
                    print("⚠️ No models found in Ollama")
                    print("   Pull a model with: ollama pull mistral:7b")
                    llm_wrapper = None
                    USE_LOCAL_LLM = False
            else:
                print("⚠️ Ollama is installed but not running")
                print("   Start it with: ollama serve")
                llm_wrapper = None
                USE_LOCAL_LLM = False
        except (subprocess.TimeoutExpired, FileNotFoundError):
            print("⚠️ Ollama not found or not running")
            print("   Install from: https://ollama.ai")
            print("   Then run: ollama pull mistral:7b && ollama serve")
            llm_wrapper = None
            USE_LOCAL_LLM = False
    except ImportError as e:
        print(f"⚠️ Error importing CrewAI LLM: {e}")
        llm_wrapper = None
        USE_LOCAL_LLM = False

# ============================================================================
# Fallback: If provider not recognized
# ============================================================================
else:
    print(f"❌ Unknown LLM provider: {LLM_PROVIDER}")
    print("   Valid options: 'ollama' or 'gemini'")
    print("   Please set LLM_PROVIDER to 'ollama' or 'gemini'")
    llm_wrapper = None
    USE_LOCAL_LLM = False

# ============================================================================
# Final Status Check
# ============================================================================
print("\n" + "="*70)
if USE_LOCAL_LLM:
    print(f"✅ LLM Configuration Complete!")
    print(f"   Provider: {LLM_PROVIDER.upper()}")
    if LLM_PROVIDER.lower() == "gemini":
        print(f"   Type: Google Gemini (Cloud)")
    elif LLM_PROVIDER.lower() == "ollama":
        print(f"   Type: Ollama (Local)")
    print("   Ready to create agents!")
else:
    print("⚠️ LLM Configuration Failed!")
    print("   Agents may not work without a configured LLM")
    if LLM_PROVIDER.lower() == "gemini":
        print("   Check: Is GEMINI_API_KEY set in .env file?")
    elif LLM_PROVIDER.lower() == "ollama":
        print("   Check: Is Ollama installed and running?")
print("="*70)


🔧 Selected LLM Provider: GEMINI


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


✅ Gemini LLM configured successfully!
   Model: gemini/gemini-2.5-flash
   API Key: Found (GEMINI_API_KEY)
   Provider: Google Gemini (Cloud)

✅ LLM Configuration Complete!
   Provider: GEMINI
   Type: Google Gemini (Cloud)
   Ready to create agents!


### Step 3.1: Define Agents

Each agent needs:
- **role**: A clear, specific role description
- **goal**: What the agent is trying to achieve
- **backstory**: Context that helps the agent understand its purpose
- **llm**: (Optional) The language model to use
- **allow_delegation**: (Optional) Whether the agent can delegate tasks to others


In [3]:
from crewai import Agent
import os

# ⚠️ IMPORTANT: Check if we have a local LLM configured
if not USE_LOCAL_LLM:
    print("⚠️ WARNING: No local LLM configured!")
    print("   CrewAI may try to use OpenAI API if OPENAI_API_KEY is set.")
    print("   To avoid API calls, ensure:")
    print("   1. Model file exists at one of:")
    print("      - ./models/mistral-7b-instruct-v0.1.Q4_K_M.gguf")
    print("      - ./src/models/mistral-7b-instruct-v0.1.Q4_K_M.gguf")
    print("   2. OPENAI_API_KEY is NOT set in environment")
    if 'OPENAI_API_KEY' in os.environ:
        print(f"   ⚠️ OPENAI_API_KEY is currently set - CrewAI will use OpenAI!")
        print("   Unset it with: unset OPENAI_API_KEY")
else:
    print(f"✅ Using local LLM: {type(llm_wrapper).__name__}")

# Agent 1: Market Research Analyst
# This agent specializes in competitive analysis and market intelligence
researcher = Agent(
    role='Market Research Analyst',
    goal='Analyze competitors and summarize their marketing strategies',
    backstory='''You are an expert in market intelligence and competitive analysis. 
    You have years of experience identifying market trends, analyzing competitor strategies, 
    and providing actionable insights. You excel at breaking down complex market data into 
    clear, understandable summaries.''',
    llm=llm_wrapper if USE_LOCAL_LLM else None,  # Use local LLM if available
    allow_delegation=False,  # This agent works independently
    verbose=True  # Show detailed reasoning
)

# Agent 2: Content Strategist
# This agent creates marketing strategies based on research
writer = Agent(
    role='Content Strategist',
    goal='Use research to create a compelling marketing strategy document',
    backstory='''You are a seasoned content strategist with a flair for storytelling. 
    You excel at transforming research data into actionable marketing strategies. 
    Your strategies are always data-driven, creative, and aligned with business objectives. 
    You have a track record of creating successful campaigns that drive engagement and growth.''',
    llm=llm_wrapper if USE_LOCAL_LLM else None,
    allow_delegation=False,
    verbose=True
)

print("✅ Agents created successfully!")
print(f"   - {researcher.role}")
print(f"   - {writer.role}")
if USE_LOCAL_LLM:
    print(f"   - Both agents using local LLM")
else:
    print(f"   - ⚠️ Agents may use default LLM (could be OpenAI)")


✅ Using local LLM: GeminiCompletion
✅ Agents created successfully!
   - Market Research Analyst
   - Content Strategist
   - Both agents using local LLM


### Step 3.2: Define Tasks

Tasks are the work units that agents perform. Key properties:
- **description**: What needs to be done (be specific!)
- **agent**: Which agent is responsible
- **expected_output**: What format/type of output is expected
- **context**: (Optional) List of tasks whose outputs will be used as input for this task

**💡 Pro Tip**: Clear task descriptions lead to better results. Be specific about what you want!


In [4]:
from crewai import Task

# Task 1: Market Research
# This task runs first and provides data for the strategy task
task1 = Task(
    description="""Analyze the top 5 competitors in the AI agent development tools market. 
    For each competitor, identify:
    1. Their primary marketing channels
    2. Key messaging and value propositions
    3. Content marketing strategies
    4. Target audience approach
    
    Focus on actionable insights that can inform our marketing strategy.""",
    agent=researcher,
    expected_output="""A structured summary containing:
    - List of 5 competitors with their names
    - For each competitor: marketing channels, messaging, content strategy, and target audience
    - Key insights and patterns across competitors"""
)

# Task 2: Strategy Creation
# This task depends on task1 - it uses the research output
task2 = Task(
    description="""Based on the competitor analysis from the Market Research Analyst, 
    create a comprehensive content marketing strategy document. The strategy should:
    1. Leverage insights from competitor analysis
    2. Identify unique positioning opportunities
    3. Propose specific content types and channels
    4. Include a high-level content calendar approach
    5. Define success metrics
    
    Make the strategy actionable and specific.""",
    agent=writer,
    expected_output="""A structured marketing strategy document with:
    - Executive summary
    - Recommended content types (blog posts, videos, webinars, etc.)
    - Channel strategy (where to publish)
    - Content calendar approach
    - Success metrics and KPIs
    - Next steps and recommendations""",
    context=[task1]  # This task uses output from task1
)

print("✅ Tasks created successfully!")
print(f"   - Task 1: {task1.description[:50]}...")
print(f"   - Task 2: {task2.description[:50]}...")
print(f"   - Task 2 uses context from Task 1: {len(task2.context) > 0 if hasattr(task2, 'context') else False}")



✅ Tasks created successfully!
   - Task 1: Analyze the top 5 competitors in the AI agent deve...
   - Task 2: Based on the competitor analysis from the Market R...
   - Task 2 uses context from Task 1: True


### Step 3.3: Create and Run the Crew

The Crew is the orchestrator that brings everything together. It:
- Manages agent coordination
- Handles task dependencies
- Executes tasks in the correct order
- Returns the final result

**Key Parameters:**
- **agents**: List of all agents in the crew
- **tasks**: List of all tasks to execute
- **verbose**: Show detailed execution logs (helpful for debugging)


In [5]:
from crewai import Crew

# Create the crew
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=True  # Show detailed execution logs
)

print("✅ Crew created successfully!")
print(f"   - Agents: {len(crew.agents)}")
print(f"   - Tasks: {len(crew.tasks)}")
print("\n🚀 Ready to execute! Run the next cell to kickoff the crew.")


✅ Crew created successfully!
   - Agents: 2
   - Tasks: 2

🚀 Ready to execute! Run the next cell to kickoff the crew.


### Step 3.4: Execute the Crew

Now let's run the crew and see the agents collaborate!


In [6]:
# Execute the crew
# This will run all tasks in the correct order based on dependencies
result = crew.kickoff()

# Display the results
print("\n" + "="*80)
print("📊 FINAL RESULTS")
print("="*80)
print(result)
print("="*80)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6ba7e04e-ad19-4516-b584-b0624516cbc9                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Analyze the top 5 competitors in the AI agent development tools market.                                  │
│      For each competitor, identify:                                                                             │
│      1. Their primary marketing channels                                                                        │
│      2. Key messaging and value propositions                                                                    │
│      3. Content marketing strategies                                                                            │
│      4. Target audience approach                                                                                │
│                                                                                                                 │
│      Focus on actionable insights that can inform our marketing strategy.                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ec5fd7c9-a2b8-4c28-8e7f-3241755f59c3                                                                     │
│  Agent: Market Research Analyst                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Strategist                                                                                      │
│                                                                                                                 │
│  Task: Based on the competitor analysis from the Market Research Analyst,                                       │
│      create a comprehensive content marketing strategy document. The strategy should:                           │
│      1. Leverage insights from competitor analysis                                                              │
│      2. Identify unique positioning opportunities                                                               │
│      3. Propose specific content types and channels                                                             │
│      4. Include a high-level content calendar approach                                                          │
│      5. Define success metrics                                                                                  │
│                                                                                                                 │
│      Make the strategy actionable and specific.                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Strategist                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Content Marketing Strategy: AgentFlow - The Reliability & Governance Platform for Production-Ready AI       │
│  Agent Orchestration                                                                                            │
│                                                                                                                 │
│  **Date:** October 26, 2023                                                                                     │
│  **Prepared For:** AgentFlow Leadership Team                                                                    │
│  **Prepared By:** [Your Name/Content Strategist]                                                                │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### 1. Executive Summary                                                                                       │
│                                                                                                                 │
│  The AI agent development tools market is experiencing rapid growth, driven primarily by open-source            │
│  frameworks (LangChain, LlamaIndex, SuperAGI, CrewAI) and large cloud providers (Microsoft Azure). While these  │
│  solutions excel at agent creation and prototyping, a significant gap exists for robust, enterprise-grade       │
│  platforms focused on the *entire lifecycle* of AI agents – specifically their deployment, monitoring,          │
│  management, and governance in production environments.                                                         │
│                                                                                                                 │
│  AgentFlow is uniquely positioned to fill this gap. Our content marketing strategy will leverage insights from  │
│  competitor success in developer-first engagement and community building, while strategically differentiating   │
│  by focusing on **reliability, security, observability, and governance** for production-ready AI agents. We     │
│  will target mid-market to enterprise developers, ML Ops engineers, and IT leaders who are moving beyond        │
│  experimentation to deploying scalable, trustworthy AI agents in business-critical applications.                │
│                                                                                                                 │
│  This document outlines a comprehensive strategy, proposing specific content types and channels, a high-level   │
│  content calendar approach, and key performance indicators (KPIs) to measure success. Our goal is to establish  │
│  AgentFlow as the indispensable partner for enterprise AI agent deployment and management, driving engagement,  │
│  adoption, and ultimately, market leadership in this crucial segment.                                           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                        


📊 FINAL RESULTS
## Content Marketing Strategy: AgentFlow - The Reliability & Governance Platform for Production-Ready AI Agent Orchestration

**Date:** October 26, 2023
**Prepared For:** AgentFlow Leadership Team
**Prepared By:** [Your Name/Content Strategist]

---

### 1. Executive Summary

The AI agent development tools market is experiencing rapid growth, driven primarily by open-source frameworks (LangChain, LlamaIndex, SuperAGI, CrewAI) and large cloud providers (Microsoft Azure). While these solutions excel at agent creation and prototyping, a significant gap exists for robust, enterprise-grade platforms focused on the *entire lifecycle* of AI agents – specifically their deployment, monitoring, management, and governance in production environments.

AgentFlow is uniquely positioned to fill this gap. Our content marketing strategy will leverage insights from competitor success in developer-first engagement and community building, while strategically differentiating by focusing on

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0fc66739-b855-4f41-95c0-982bb5b085f2                                                                     │
│  Agent: Content Strategist                                                                                      │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────── Trace Batch Finalization ──────────────────────────╮
│ ✅ Trace batch finalized with session ID:                                    │
│ 1df625c4-d909-44d3-a82c-dcae5c2fa1cf                                         │
│                                                                              │
│ 🔗 View here:                                                                │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/1df625c4-d909-44d │
│ 3-a82c-dcae5c2fa1cf?access_code=TRACE-29faa5b3ca                             │
│ 🔑 Access Code: TRACE-29faa5b3ca                                             │
╰──────────────────────────────────────────────────────────────────────────────╯


╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 6ba7e04e-ad19-4516-b584-b0624516cbc9                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## Content Marketing Strategy: AgentFlow - The Reliability & Governance Platform for             │
│  Production-Ready AI Agent Orchestration                                                                        │
│                                                                                                                 │
│  **Date:** October 26, 2023                                                                                     │
│  **Prepared For:** AgentFlow Leadership Team                                                                    │
│  **Prepared By:** [Your Name/Content Strategist]                                                                │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### 1. Executive Summary                                                                                       │
│                                                                                                                 │
│  The AI agent development tools market is experiencing rapid growth, driven primarily by open-source            │
│  frameworks (LangChain, LlamaIndex, SuperAGI, CrewAI) and large cloud providers (Microsoft Azure). While these  │
│  solutions excel at agent creation and prototyping, a significant gap exists for robust, enterprise-grade       │
│  platforms focused on the *entire lifecycle* of AI agents – specifically their deployment, monitoring,          │
│  management, and governance in production environments.                                                         │
│                                                                                                                 │
│  AgentFlow is uniquely positioned to fill this gap. Our content marketing strategy will leverage insights from  │
│  competitor success in developer-first engagement and community building, while strategically differentiating   │
│  by focusing on **reliability, security, observability, and governance** for production-ready AI agents. We     │
│  will target mid-market to enterprise developers, ML Ops engineers, and IT leaders who are moving beyond        │
│  experimentation to deploying scalable, trustworthy AI agents in business-critical applications.                │
│                                                                                                                 │
│  This document outlines a comprehensive strategy, proposing specific content types and channels, a high-level   │
│  content calendar approach, and key performance indicators (KPIs) to measure success. Our goal is to establish  │
│  AgentFlow as the indispensable partner for enterprise AI agent deployment and management, driving engagement,  │
│  adoption, and ultimately, market leadership in this crucial segment.                                           │
│                                                                                                                 │
│  ---                                                  

## 🎓 Understanding the Output

When you run the crew, you'll see:

1. **Task Execution Logs**: Each agent's reasoning process
2. **Intermediate Outputs**: Results from each task
3. **Final Output**: The combined result from all tasks

### Key Observations

- **Sequential Execution**: Task 2 waits for Task 1 to complete
- **Context Passing**: The writer agent receives the researcher's output
- **Specialized Roles**: Each agent focuses on its expertise
- **Collaborative Result**: The final output combines both agents' work

---

## 📚 Example 2: Advanced Research & Writing Crew

Now let's build a more complex system with three agents working together on a research and writing project.


### Scenario: Technical Blog Post Creation

We'll create a crew that:
1. **Researcher Agent**: Conducts in-depth research on a technical topic
2. **Technical Writer Agent**: Creates a well-structured blog post
3. **Editor Agent**: Reviews and improves the content

This demonstrates:
- Multi-step workflows
- Quality assurance through editing
- More complex task dependencies


In [7]:
# Define three specialized agents

# Agent 1: Research Specialist
researcher_agent = Agent(
    role='Technical Research Specialist',
    goal='Conduct thorough research on technical topics and compile comprehensive information',
    backstory='''You are a technical research expert with a PhD in Computer Science. 
    You excel at finding authoritative sources, understanding complex technical concepts, 
    and synthesizing information from multiple sources. You have a keen eye for detail 
    and always verify information for accuracy.''',
    llm=llm_wrapper if USE_LOCAL_LLM else None,
    allow_delegation=False,
    verbose=True
)

# Agent 2: Technical Writer
writer_agent = Agent(
    role='Technical Content Writer',
    goal='Transform research into engaging, well-structured technical blog posts',
    backstory='''You are an experienced technical writer who specializes in making complex 
    topics accessible. You have written hundreds of technical blog posts and understand 
    how to structure content for maximum readability and engagement. You balance technical 
    accuracy with clarity and always include practical examples.''',
    llm=llm_wrapper if USE_LOCAL_LLM else None,
    allow_delegation=False,
    verbose=True
)

# Agent 3: Editor
editor_agent = Agent(
    role='Technical Editor',
    goal='Review and improve technical content for clarity, accuracy, and engagement',
    backstory='''You are a senior technical editor with years of experience reviewing 
    technical content. You have an exceptional eye for detail, grammar, and style. 
    You ensure content is accurate, well-structured, and engaging. You provide constructive 
    feedback that improves content quality without losing the author's voice.''',
    llm=llm_wrapper if USE_LOCAL_LLM else None,
    allow_delegation=False,
    verbose=True
)

print("✅ Three-agent crew created!")
print(f"   - {researcher_agent.role}")
print(f"   - {writer_agent.role}")
print(f"   - {editor_agent.role}")


✅ Three-agent crew created!
   - Technical Research Specialist
   - Technical Content Writer
   - Technical Editor


In [8]:
# Define tasks with dependencies

# Task 1: Research
research_task = Task(
    description="""Research the topic: 'How Multi-Agent AI Systems Work'
    
    Provide comprehensive information on:
    1. What multi-agent systems are
    2. Key concepts and architectures
    3. Real-world applications
    4. Benefits and challenges
    5. Current trends and future directions
    
    Include specific examples and use cases. Aim for depth and accuracy.""",
    agent=researcher_agent,
    expected_output="""A comprehensive research document with:
    - Definition and explanation of multi-agent systems
    - Key concepts and terminology
    - Architecture patterns
    - Real-world applications with examples
    - Benefits and challenges
    - Current trends and future outlook
    - References to key concepts"""
)

# Task 2: Writing (depends on research)
writing_task = Task(
    description="""Using the research provided, write a technical blog post titled 
    'Understanding Multi-Agent AI Systems: A Complete Guide'
    
    The blog post should:
    1. Be 1500-2000 words
    2. Have a clear introduction that hooks the reader
    3. Include sections with headers
    4. Use examples and analogies to explain complex concepts
    5. Include a conclusion that summarizes key points
    6. Be written in an engaging, accessible style
    
    Structure:
    - Introduction
    - What are Multi-Agent Systems?
    - How They Work (Architecture)
    - Real-World Applications
    - Benefits and Challenges
    - Future Outlook
    - Conclusion""",
    agent=writer_agent,
    expected_output="""A complete blog post with:
    - Engaging introduction
    - Well-structured sections with headers
    - Clear explanations with examples
    - Proper conclusion
    - Approximately 1500-2000 words""",
    context=[research_task]
)

# Task 3: Editing (uses context from writing)
editing_task = Task(
    description="""Review and edit the blog post to ensure:
    1. Technical accuracy
    2. Clarity and readability
    3. Proper grammar and style
    4. Logical flow between sections
    5. Engaging tone throughout
    6. No redundancy or unnecessary content
    
    Provide the final polished version with any improvements made.""",
    agent=editor_agent,
    expected_output="""A polished, edited version of the blog post with:
    - All improvements made
    - Clear indication of what was changed (if any)
    - Final version ready for publication""",
    context=[writing_task]
)

print("✅ Tasks created with dependencies!")
print(f"   Research → Writing → Editing")


✅ Tasks created with dependencies!
   Research → Writing → Editing


In [9]:
# Create and run the research crew
research_crew = Crew(
    agents=[researcher_agent, writer_agent, editor_agent],
    tasks=[research_task, writing_task, editing_task],
    verbose=True
)

print("✅ Research crew ready!")
print("🚀 Execute the next cell to run the crew...")


✅ Research crew ready!
🚀 Execute the next cell to run the crew...


In [10]:
# Uncomment to run the research crew
# This may take several minutes depending on your LLM setup

result = research_crew.kickoff()

print("\n" + "="*80)
print("📝 FINAL BLOG POST")
print("="*80)
print(result)
print("="*80)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8f11ddda-a04e-40f5-a7f8-f8ab6dbbe763                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Research Specialist                                                                           │
│                                                                                                                 │
│  Task: Research the topic: 'How Multi-Agent AI Systems Work'                                                    │
│                                                                                                                 │
│      Provide comprehensive information on:                                                                      │
│      1. What multi-agent systems are                                                                            │
│      2. Key concepts and architectures                                                                          │
│      3. Real-world applications                                                                                 │
│      4. Benefits and challenges                                                                                 │
│      5. Current trends and future directions                                                                    │
│                                                                                                                 │
│      Include specific examples and use cases. Aim for depth and accuracy.                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Research Specialist                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## How Multi-Agent AI Systems Work: A Comprehensive Research Document                                          │
│                                                                                                                 │
│  ### 1. What Multi-Agent Systems Are                                                                            │
│                                                                                                                 │
│  A **Multi-Agent System (MAS)** is a system composed of multiple interacting intelligent agents within an       │
│  environment. These agents are autonomous entities capable of perceiving their environment, making decisions,   │
│  and performing actions to achieve their individual or collective goals. Unlike a single, monolithic AI         │
│  system, MAS distribute intelligence and tasks across several specialized or general-purpose agents, allowing   │
│  for more robust, scalable, and flexible solutions to complex problems.                                         │
│                                                                                                                 │
│  **Key Characteristics of Agents in MAS:**                                                                      │
│  *   **Autonomy:** Agents operate without direct human or external intervention, having control over their      │
│  internal state and behavior.                                                                                   │
│  *   **Pro-activeness:** Agents are goal-directed and take initiative to achieve their objectives, rather than  │
│  merely reacting to the environment.                                                                            │
│  *   **Reactivity:** Agents perceive their environment and respond in a timely fashion to changes that occur    │
│  within it.                                                                                                     │
│  *   **Social Ability:** Agents interact with other agents (and potentially humans) through communication,      │
│  cooperation, coordination, and negotiation to achieve their goals.                                             │
│                                                                                                                 │
│  **Distinction from Single-Agent Systems:**                                                                     │
│  In a single-agent system, one AI entity is responsible for all decision-making and actions. This can lead to   │
│  bottlenecks, single points of failure, and difficulty in scaling for complex, distributed problems. MAS        │
│  overcome these limitations by distributing intelligence and labor, mirroring how complex problems are often    │
│  solved in human societies through collaboration and specialization.                                            │
│                                                                                                                 │
│  ### 2. Key Concepts and Architectures                                                                          │
│                                                                                                                 │
│  Understanding the internal workings of MAS requires delving into the properties of agents, their interaction   │
│  mechanisms, and the structural patterns that govern th

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: df93fc02-79bc-41d8-97dd-4a018827d816                                                                     │
│  Agent: Technical Research Specialist                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Editor                                                                                        │
│                                                                                                                 │
│  Task: Review and edit the blog post to ensure:                                                                 │
│      1. Technical accuracy                                                                                      │
│      2. Clarity and readability                                                                                 │
│      3. Proper grammar and style                                                                                │
│      4. Logical flow between sections                                                                           │
│      5. Engaging tone throughout                                                                                │
│      6. No redundancy or unnecessary content                                                                    │
│                                                                                                                 │
│      Provide the final polished version with any improvements made.                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1fd8c0b1-9a84-4c77-afb3-3f12aa19ba22                                                                     │
│  Agent: Technical Editor                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────── Trace Batch Finalization ──────────────────────────╮
│ ✅ Trace batch finalized with session ID:                                    │
│ 630c1078-fc20-4f35-a3e5-fb2ff4ae3883                                         │
│                                                                              │
│ 🔗 View here:                                                                │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/630c1078-fc20-4f3 │
│ 5-a3e5-fb2ff4ae3883?access_code=TRACE-31bd2bff3a                             │
│ 🔑 Access Code: TRACE-31bd2bff3a                                             │
╰──────────────────────────────────────────────────────────────────────────────╯


╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8f11ddda-a04e-40f5-a7f8-f8ab6dbbe763                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Understanding Multi-Agent AI Systems: A Complete Guide                                         │
│                                                                                                                 │
│  Imagine a world where complex problems aren't solved by a single, monolithic super-brain, but by a symphony    │
│  of specialized intelligences. Each contributes its unique expertise, collaborating and even negotiating to     │
│  achieve a common goal. This isn't science fiction; it's the rapidly evolving reality of Multi-Agent AI         │
│  Systems (MAS).                                                                                                 │
│                                                                                                                 │
│  In today's interconnected and dynamic world, relying on a single AI to manage everything from smart cities to  │
│  disaster response is often impractical, inefficient, and prone to failure. Just as a human organization        │
│  benefits from diverse teams and delegated responsibilities, AI systems gain immense power and resilience when  │
│  designed as a collective of interacting agents.                                                                │
│                                                                                                                 │
│  This guide will take you on a comprehensive journey through the fascinating landscape of Multi-Agent Systems.  │
│  We'll demystify what they are, how they operate, explore their groundbreaking applications, weigh their        │
│  benefits against their challenges, and peer into the exciting future they promise. If you've ever wondered     │
│  how AI can tackle truly grand challenges, distributed intelligence is your answer.                             │
│                                                                                                                 │
│  ## What Are Multi-Agent Systems?                                                                               │
│                                                                                                                 │
│  At its core, a **Multi-Agent System (MAS)** is a sophisticated AI framework comprising multiple intelligent    │
│  entities, known as "agents," that interact within a shared environment. Think of it like a highly organized    │
│  ant colony or a well-coordinated sports team, but with each member being an autonomous AI. Unlike a single,    │
│  all-encompassing AI, MAS distribute intelligence and tasks among several agents, allowing for more robust,     │
│  scalable, and flexible solutions to problems that would overwhelm any solitary intelligence.                   │
│                                                                                                                 │
│  To truly grasp MAS, it's essential to understand the defining characteristics of these individual agents:      │
│                                                                                                                 │
│  *   **Autonomy:** Agents operate independently, making decisions and executing actions without constant human  │
│  or external oversight. They are masters of their own 

## 🔧 Example 3: Code Review Crew

Let's build a more advanced system for code quality assurance with multiple specialized agents.


### Scenario: Automated Code Review System

This crew will:
1. **Code Analyzer**: Analyze code for bugs and issues
2. **Security Specialist**: Check for security vulnerabilities
3. **Performance Expert**: Review code for performance issues
4. **Documentation Reviewer**: Ensure code is well-documented
5. **Review Coordinator**: Synthesize all reviews into a final report

This demonstrates:
- Parallel task execution (some tasks can run simultaneously)
- Complex coordination
- Multiple specialized perspectives


In [11]:
# Sample code to review (you can replace this with your own code)
sample_code = """
def process_user_data(user_data):
    # Process user data
    result = []
    for item in user_data:
        if item['age'] > 18:
            result.append(item['name'])
    return result

def authenticate_user(username, password):
    # Simple authentication
    if username == 'admin' and password == 'password123':
        return True
    return False

def fetch_data(url):
    import requests
    response = requests.get(url)
    return response.json()
"""

print("📝 Sample code to review:")
print(sample_code)


📝 Sample code to review:

def process_user_data(user_data):
    # Process user data
    result = []
    for item in user_data:
        if item['age'] > 18:
            result.append(item['name'])
    return result

def authenticate_user(username, password):
    # Simple authentication
    if username == 'admin' and password == 'password123':
        return True
    return False

def fetch_data(url):
    import requests
    response = requests.get(url)
    return response.json()



In [12]:
# Define specialized code review agents

code_analyzer = Agent(
    role='Senior Code Analyst',
    goal='Identify bugs, code smells, and potential issues in code',
    backstory='''You are a senior software engineer with 15+ years of experience. 
    You have an exceptional ability to spot bugs, code smells, and potential issues. 
    You understand best practices and coding standards across multiple languages.''',
    llm=llm_wrapper if USE_LOCAL_LLM else None,
    allow_delegation=False,
    verbose=True
)

security_specialist = Agent(
    role='Security Expert',
    goal='Identify security vulnerabilities and suggest secure coding practices',
    backstory='''You are a cybersecurity expert specializing in secure coding practices. 
    You have extensive experience identifying vulnerabilities like SQL injection, XSS, 
    authentication flaws, and data exposure issues. You stay updated with the latest 
    security threats and mitigation strategies.''',
    llm=llm_wrapper if USE_LOCAL_LLM else None,
    allow_delegation=False,
    verbose=True
)

performance_expert = Agent(
    role='Performance Optimization Specialist',
    goal='Identify performance bottlenecks and suggest optimizations',
    backstory='''You are a performance engineering expert. You specialize in identifying 
    bottlenecks, memory leaks, inefficient algorithms, and optimization opportunities. 
    You understand how to write efficient, scalable code.''',
    llm=llm_wrapper if USE_LOCAL_LLM else None,
    allow_delegation=False,
    verbose=True
)

doc_reviewer = Agent(
    role='Technical Documentation Specialist',
    goal='Review code documentation and suggest improvements',
    backstory='''You are a technical writing expert who specializes in code documentation. 
    You understand the importance of clear docstrings, comments, and README files. 
    You ensure code is maintainable and understandable for future developers.''',
    llm=llm_wrapper if USE_LOCAL_LLM else None,
    allow_delegation=False,
    verbose=True
)

review_coordinator = Agent(
    role='Code Review Coordinator',
    goal='Synthesize all code reviews into a comprehensive report',
    backstory='''You are a technical lead who specializes in coordinating code reviews. 
    You excel at synthesizing feedback from multiple reviewers, prioritizing issues, 
    and creating actionable improvement plans. You ensure all feedback is clear and constructive.''',
    llm=llm_wrapper if USE_LOCAL_LLM else None,
    allow_delegation=False,
    verbose=True
)

print("✅ Code review agents created!")


✅ Code review agents created!


In [13]:
# Define review tasks
# Note: Some tasks can run in parallel (no dependencies)

bug_analysis_task = Task(
    description=f"""Analyze the following Python code for bugs, code smells, and potential issues:

{sample_code}

Provide:
1. List of bugs found
2. Code smells identified
3. Potential runtime errors
4. Suggestions for improvement
5. Code quality rating (1-10)""",
    agent=code_analyzer,
    expected_output="A detailed bug analysis report with specific issues and recommendations"
)

security_review_task = Task(
    description=f"""Review the following code for security vulnerabilities:

{sample_code}

Check for:
1. Authentication/authorization issues
2. Input validation problems
3. Data exposure risks
4. Insecure dependencies or practices
5. OWASP Top 10 vulnerabilities

Provide specific security recommendations.""",
    agent=security_specialist,
    expected_output="A security review report with vulnerabilities and mitigation strategies"
)

performance_review_task = Task(
    description=f"""Analyze the following code for performance issues:

{sample_code}

Check for:
1. Algorithmic inefficiencies
2. Memory usage problems
3. Unnecessary computations
4. Scalability concerns
5. Optimization opportunities

Provide specific performance recommendations.""",
    agent=performance_expert,
    expected_output="A performance analysis report with bottlenecks and optimization suggestions"
)

documentation_review_task = Task(
    description=f"""Review the documentation quality of the following code:

{sample_code}

Evaluate:
1. Function docstrings
2. Code comments
3. Variable naming clarity
4. Overall code readability
5. Missing documentation

Provide specific documentation improvements.""",
    agent=doc_reviewer,
    expected_output="A documentation review report with specific improvement suggestions"
)

# Final synthesis task (depends on all review tasks)
final_report_task = Task(
    description="""Synthesize all code review reports into a comprehensive final report.
    
    The report should include:
    1. Executive summary
    2. Critical issues (high priority)
    3. Important issues (medium priority)
    4. Nice-to-have improvements (low priority)
    5. Overall code quality assessment
    6. Actionable improvement plan
    7. Priority-ordered recommendations
    
    Make the report clear, actionable, and prioritized.""",
    agent=review_coordinator,
    expected_output="A comprehensive code review report with prioritized recommendations",
    context=[bug_analysis_task, security_review_task, performance_review_task, documentation_review_task]
)

print("✅ Code review tasks created!")
print("   - 4 parallel review tasks")
print("   - 1 synthesis task (uses context from all reviews)")


✅ Code review tasks created!
   - 4 parallel review tasks
   - 1 synthesis task (uses context from all reviews)


In [14]:
# Create the code review crew
code_review_crew = Crew(
    agents=[code_analyzer, security_specialist, performance_expert, doc_reviewer, review_coordinator],
    tasks=[bug_analysis_task, security_review_task, performance_review_task, documentation_review_task, final_report_task],
    verbose=True
)

print("✅ Code review crew ready!")
print("🚀 Execute the next cell to run the crew...")


✅ Code review crew ready!
🚀 Execute the next cell to run the crew...


In [ ]:
# Uncomment to run the code review crew
# This will execute all review tasks and create a final report

result = code_review_crew.kickoff()

print("\n" + "="*80)
print("📋 FINAL CODE REVIEW REPORT")
print("="*80)
print(result)
print("="*80)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 548a74cc-8307-4f68-a19d-0894b95c4c7b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Code Analyst                                                                                     │
│                                                                                                                 │
│  Task: Analyze the following Python code for bugs, code smells, and potential issues:                           │
│                                                                                                                 │
│                                                                                                                 │
│  def process_user_data(user_data):                                                                              │
│      # Process user data                                                                                        │
│      result = []                                                                                                │
│      for item in user_data:                                                                                     │
│          if item['age'] > 18:                                                                                   │
│              result.append(item['name'])                                                                        │
│      return result                                                                                              │
│                                                                                                                 │
│  def authenticate_user(username, password):                                                                     │
│      # Simple authentication                                                                                    │
│      if username == 'admin' and password == 'password123':                                                      │
│          return True                                                                                            │
│      return False                                                                                               │
│                                                                                                                 │
│  def fetch_data(url):                                                                                           │
│      import requests                                                                                            │
│      response = requests.get(url)                                                                               │
│      return response.json()                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│  Provide:                                                                                                       │
│  1. List of bugs found                                                                                          │
│  2. Code smells identified                                                                                      │
│  3. Potential runtime errors                                                                                    │
│  4. Suggestions for improvement                                                                                 │
│  5. Code quality rating (1-10)                                                                                  │
│                                                                                                                 │
╰────────────────────────────────────────────────────────

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Code Analyst                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Here is a detailed analysis of the provided Python code, identifying bugs, code smells, potential runtime      │
│  errors, and suggestions for improvement, along with a code quality rating.                                     │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **Detailed Bug Analysis Report**                                                                           │
│                                                                                                                 │
│  **1. List of bugs found**                                                                                      │
│                                                                                                                 │
│  *   **`authenticate_user` Function - Critical Security Vulnerability**:                                        │
│      *   **Issue**: The function uses hardcoded username (`'admin'`) and password (`'password123'`) for         │
│  authentication.                                                                                                │
│      *   **Impact**: This is a severe security flaw. In any real-world application, hardcoded credentials make  │
│  the system trivial to compromise. Anyone with access to the codebase (or who can reverse-engineer the          │
│  application) can gain unauthorized access. This is a fundamental breach of security best practices.            │
│                                                                                                                 │
│  **2. Code smells identified**                                                                                  │
│                                                                                                                 │
│  *   **`process_user_data` Function**:                                                                          │
│      *   **Lack of Input Validation**: The function assumes `user_data` will always be an iterable of           │
│  dictionaries, and that each dictionary will contain 'age' and 'name' keys with appropriate data types. There   │
│  are no explicit checks, making the function brittle to malformed input.                                        │
│      *   **Magic Number**: The value `18` is used directly in the comparison `item['age'] > 18`. While common   │
│  for age, defining it as a named constant (e.g., `MIN_ADULT_AGE = 18`) would improve readability and            │
│  maintainability.                                                                                               │
│  *   **`authenticate_user` Function**:                                                                          │
│      *   **Hardcoded Credentials**: (As noted in bugs, this is also a major code smell.) Storing or comparing   │
│  credentials directly in code is a serious security anti-pattern and makes the system vulnerable.               │
│      *   **Lack of Secure Password Handling**: Passwords should never be stored or compared in plain text.      │
│  This indicates a complete absence of proper password h

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Code Review Coordinator                                                                                 │
│                                                                                                                 │
│  Task: Synthesize all code review reports into a comprehensive final report.                                    │
│                                                                                                                 │
│      The report should include:                                                                                 │
│      1. Executive summary                                                                                       │
│      2. Critical issues (high priority)                                                                         │
│      3. Important issues (medium priority)                                                                      │
│      4. Nice-to-have improvements (low priority)                                                                │
│      5. Overall code quality assessment                                                                         │
│      6. Actionable improvement plan                                                                             │
│      7. Priority-ordered recommendations                                                                        │
│                                                                                                                 │
│      Make the report clear, actionable, and prioritized.                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bc005ca6-60ed-4ce0-bc2e-f7475bb1062f                                                                     │
│  Agent: Code Review Coordinator                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────── Trace Batch Finalization ──────────────────────────╮
│ ✅ Trace batch finalized with session ID:                                    │
│ 3a30fa1a-c7ba-49d5-adcd-90c926817392                                         │
│                                                                              │
│ 🔗 View here:                                                                │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/3a30fa1a-c7ba-49d │
│ 5-adcd-90c926817392?access_code=TRACE-caa203338a                             │
│ 🔑 Access Code: TRACE-caa203338a                                             │
╰──────────────────────────────────────────────────────────────────────────────╯


╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 548a74cc-8307-4f68-a19d-0894b95c4c7b                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## Comprehensive Code Review Report                                                              │
│                                                                                                                 │
│  ### 1. Executive Summary                                                                                       │
│                                                                                                                 │
│  This report synthesizes the findings from a detailed analysis of the provided Python codebase, revealing       │
│  significant deficiencies across security, reliability, performance, and documentation. The most critical       │
│  finding is a severe security vulnerability within the `authenticate_user` function, which employs hardcoded    │
│  credentials, rendering the application immediately compromised and unsuitable for any real-world deployment.   │
│                                                                                                                 │
│  Furthermore, the `fetch_data` function suffers from a complete absence of error handling and timeouts for      │
│  network requests, making the application highly unstable and prone to crashes. Performance is unnecessarily    │
│  degraded by inefficient module imports, and the `process_user_data` function lacks robust input validation,    │
│  leading to brittle code. The overall lack of comprehensive documentation, including docstrings and type        │
│  hints, severely hinders maintainability, understandability, and future development.                            │
│                                                                                                                 │
│  Based on these findings, the code quality is assessed as critically low (2/10), demanding immediate and        │
│  comprehensive remediation. The following report outlines specific issues, categorized by priority, and         │
│  provides an actionable improvement plan to address these critical shortcomings.                                │
│                                                                                                                 │
│  ### 2. Critical Issues (High Priority)                                                                         │
│                                                                                                                 │
│  These issues represent immediate threats to the application's security, stability, or fundamental              │
│  functionality and require urgent attention.                                                                    │
│                                                                                                                 │
│  *   **Authentication Security Vulnerability (CVE-CRITICAL-001):**                                              │
│      *   **Issue:** The `authenticate_user` function uses hardcoded username (`'admin'`) and password           │
│  (`'password123'`) for authentication.                                                                          │
│      *   **Impact:** This is a severe security flaw allowing trivial compromise of the system. Anyone with      │
│  access to the codebase can gain unauthorized access. 

## 🎯 Key Concepts Learned So Far

### 1. Agent Definition
- **Role**: Defines what the agent does
- **Goal**: What the agent aims to achieve
- **Backstory**: Provides context and personality
- **LLM**: The language model powering the agent
- **Allow Delegation**: Whether agents can delegate tasks

### 2. Task Management
- **Description**: Clear, specific instructions
- **Agent Assignment**: Which agent handles the task
- **Context**: List of tasks whose outputs will be used as input
- **Expected Output**: Format and content expectations

### 3. Crew Orchestration
- **Sequential Execution**: Tasks run in order based on context
- **Parallel Execution**: Independent tasks (no context) can run simultaneously
- **Context Passing**: Outputs flow from context tasks to dependent tasks
- **Final Synthesis**: Combining all results

### 4. Best Practices
- ✅ Be specific in task descriptions
- ✅ Define clear expected outputs
- ✅ Use context to control flow and pass information between tasks
- ✅ Give agents detailed backstories
- ✅ Set verbose=True for debugging

---

## 🏋️ Practice Exercises

Now it's your turn! Try these exercises to reinforce your learning.


### Exercise 1: Product Launch Crew (Beginner)

**Objective**: Create a simple two-agent crew for product launch planning.

**Requirements**:
1. Create a **Product Analyst** agent that analyzes market opportunities
2. Create a **Launch Strategist** agent that creates a launch plan
3. The strategist should use the analyst's research
4. Focus on a specific product (e.g., "AI-powered task management app")

**Expected Output**: A market analysis and launch strategy document

**Hints**:
- Start with clear role definitions
- Make task descriptions specific
- Use task dependencies correctly

**Solution Template**:
```python
# Your code here
analyst = Agent(...)
strategist = Agent(...)
# ... create tasks and crew
```


In [ ]:
# Exercise 1: Product Launch Crew
# Write your solution here

# TODO: Define agents
# TODO: Define tasks
# TODO: Create and run crew


### Exercise 2: Customer Support Crew (Intermediate)

**Objective**: Build a three-agent system for handling customer support inquiries.

**Requirements**:
1. **Support Triage Agent**: Categorizes and prioritizes support tickets
2. **Technical Specialist Agent**: Provides technical solutions
3. **Response Writer Agent**: Creates customer-friendly responses

**Scenario**: A customer reports "My app crashes when I try to upload a large file"

**Expected Output**: 
- Ticket categorization
- Technical analysis
- Customer-friendly response

**Challenge**: Make the response writer use both the triage and technical analysis.


In [ ]:
# Exercise 2: Customer Support Crew
# Write your solution here

customer_issue = "My app crashes when I try to upload a large file"

# TODO: Create three agents
# TODO: Create tasks with proper dependencies
# TODO: Create and run crew


### Exercise 3: Data Analysis Crew (Advanced)

**Objective**: Create a multi-agent system for data analysis and reporting.

**Requirements**:
1. **Data Analyst Agent**: Analyzes data and identifies patterns
2. **Statistician Agent**: Performs statistical analysis
3. **Visualization Specialist Agent**: Suggests visualizations
4. **Report Writer Agent**: Creates a comprehensive report

**Scenario**: Analyze a dataset (you can use a simple CSV or create synthetic data)

**Expected Output**: 
- Data insights
- Statistical findings
- Visualization recommendations
- Complete analysis report

**Challenge**: 
- Make some tasks run in parallel
- Have the report writer synthesize all inputs
- Include specific data in your task descriptions


In [ ]:
# Exercise 3: Data Analysis Crew
# Write your solution here

# Sample data (you can replace with real data)
sample_data = """
Product, Sales, Region, Month
Widget A, 1500, North, January
Widget B, 2300, South, January
Widget A, 1800, North, February
Widget B, 2100, South, February
Widget A, 2000, North, March
Widget B, 2500, South, March
"""

# TODO: Create four specialized agents
# TODO: Create tasks (some can run in parallel)
# TODO: Create final synthesis task
# TODO: Create and run crew


### Exercise 4: Content Marketing Campaign (Advanced)

**Objective**: Build a comprehensive content marketing crew with 5+ agents.

**Requirements**:
1. **Market Researcher**: Analyzes target audience and competitors
2. **Content Strategist**: Creates content strategy
3. **SEO Specialist**: Optimizes for search engines
4. **Social Media Expert**: Plans social media approach
5. **Content Creator**: Writes the actual content
6. **Campaign Manager**: Coordinates and synthesizes everything

**Expected Output**: A complete content marketing campaign plan

**Challenge**: 
- Use complex dependencies
- Have multiple parallel tasks
- Create a comprehensive final synthesis


In [ ]:
# Exercise 4: Content Marketing Campaign
# Write your solution here

campaign_topic = "Promoting a new AI-powered productivity tool"

# TODO: Create 6 specialized agents
# TODO: Design task workflow with dependencies
# TODO: Create and run crew


## 🚀 Advanced Features

Now let's explore some advanced CrewAI features that make your agents more powerful.


### Feature 1: Agent Memory

Agents can remember past interactions, allowing them to build context over time.

**Use Cases**:
- Long-running conversations
- Learning from previous tasks
- Building on past insights

**Example**:
```python
agent_with_memory = Agent(
    role='Research Assistant',
    goal='Conduct research',
    backstory='...',
    memory=True,  # Enable memory
    max_iter=5,   # Maximum iterations
    verbose=True
)
```


### Feature 2: Agent Tools

Agents can use external tools to extend their capabilities.

**Common Tools**:
- Web search
- File operations
- API calls
- Database queries
- Code execution

**Example**:
```python
from crewai_tools import SerperDevTool, WebsiteSearchTool

# Agent with tools
agent = Agent(
    role='Researcher',
    goal='Research topics',
    backstory='...',
    tools=[SerperDevTool(), WebsiteSearchTool()],
    verbose=True
)
```

**Note**: You may need to install additional packages for tools:
```bash
pip install crewai-tools
```


### Feature 3: Task Output Formatting

You can specify exact output formats for tasks.

**Example**:
```python
task = Task(
    description="Analyze market trends",
    agent=analyst,
    expected_output="""A JSON object with:
    {
        "trends": ["trend1", "trend2"],
        "insights": "summary",
        "recommendations": ["rec1", "rec2"]
    }"""
)
```


### Feature 4: Crew Configuration

Crews can be configured with various options.

**Example**:
```python
crew = Crew(
    agents=[agent1, agent2],
    tasks=[task1, task2],
    verbose=True,           # Show detailed logs
    max_iter=15,           # Maximum iterations
    memory=True,           # Enable crew-level memory
    process=Process.sequential  # or Process.hierarchical
)
```


## 💡 Best Practices & Tips

### 1. Agent Design
- ✅ **Be Specific**: Clear roles lead to better results
- ✅ **Detailed Backstories**: Help agents understand their purpose
- ✅ **Appropriate Goals**: Align goals with desired outcomes
- ❌ **Avoid Overlap**: Each agent should have a unique role

### 2. Task Design
- ✅ **Clear Descriptions**: Be explicit about what you want
- ✅ **Specific Outputs**: Define expected output format
- ✅ **Logical Dependencies**: Use dependencies to control flow
- ✅ **Appropriate Scope**: Don't make tasks too broad or too narrow

### 3. Crew Management
- ✅ **Start Simple**: Begin with 2-3 agents, then scale up
- ✅ **Test Incrementally**: Test each agent individually first
- ✅ **Use Verbose Mode**: Helps debug and understand agent reasoning
- ✅ **Monitor Performance**: Track execution time and quality

### 4. LLM Selection
- ✅ **Cloud LLMs**: Better for production, require API keys
- ✅ **Local LLMs**: Better for privacy, require more resources
- ✅ **Model Size**: Larger models = better quality but slower
- ✅ **Temperature**: Lower (0.1-0.3) for factual tasks, higher (0.7-0.9) for creative

### 5. Common Pitfalls
- ❌ **Vague Task Descriptions**: Leads to poor results
- ❌ **Circular Dependencies**: Tasks depending on each other
- ❌ **Too Many Agents**: Can slow down execution
- ❌ **Ignoring Output Format**: Unstructured outputs are hard to use


## 🔍 Debugging Tips

### 1. Enable Verbose Mode
Always set `verbose=True` when developing to see agent reasoning:
```python
agent = Agent(..., verbose=True)
crew = Crew(..., verbose=True)
```

### 2. Test Agents Individually
Create simple test tasks to verify each agent works:
```python
test_task = Task(
    description="Simple test task",
    agent=your_agent,
    expected_output="Test output"
)
test_crew = Crew(agents=[your_agent], tasks=[test_task], verbose=True)
result = test_crew.kickoff()
```

### 3. Check Task Context
Verify context (dependencies) are set correctly:
```python
print(f"Task 2 uses context from: {[t.description for t in task2.context]}")
```

### 4. Inspect Intermediate Outputs
CrewAI provides access to task outputs:
```python
result = crew.kickoff()
# Access individual task results if needed
```

### 5. Common Issues

**Issue**: Agents produce generic outputs
- **Solution**: Make task descriptions more specific, add examples

**Issue**: Tasks execute in wrong order
- **Solution**: Check context parameter, ensure tasks are properly linked

**Issue**: Agents don't use previous task outputs
- **Solution**: Verify context parameter, check task descriptions reference previous work

**Issue**: Execution is too slow
- **Solution**: Use smaller models, reduce task complexity, enable parallel execution where possible


## 📚 Additional Resources

### Official Documentation
- **CrewAI Docs**: https://docs.crewai.com
- **GitHub Repository**: https://github.com/joaomdmoura/crewAI
- **Examples**: Check the CrewAI GitHub for more examples

### Related Topics to Explore
1. **LangChain**: CrewAI is built on LangChain - learn more about it
2. **Multi-Agent Systems**: Research academic papers on MAS
3. **Agent Orchestration**: Learn about different orchestration patterns
4. **Prompt Engineering**: Improve your agent prompts for better results

### Next Steps
1. ✅ Complete all practice exercises
2. ✅ Build your own custom crew for a real project
3. ✅ Experiment with different LLM providers
4. ✅ Try adding tools to your agents
5. ✅ Explore CrewAI's advanced features

---

## 🎉 Conclusion

Congratulations! You've learned how to:
- ✅ Create AI agents with CrewAI
- ✅ Define tasks and manage dependencies
- ✅ Orchestrate multi-agent systems
- ✅ Build complex workflows
- ✅ Apply best practices

### Key Takeaways

1. **CrewAI simplifies multi-agent orchestration** - No need to manually coordinate agents
2. **Clear definitions matter** - Good roles, goals, and backstories lead to better results
3. **Dependencies control flow** - Use them to create logical workflows
4. **Start simple, then scale** - Begin with 2-3 agents, add complexity gradually
5. **Practice makes perfect** - Build multiple crews to master the concepts

### Your Journey Continues

Now that you understand CrewAI fundamentals:
- Build real-world projects
- Experiment with different agent configurations
- Explore advanced features like tools and memory
- Contribute to the CrewAI community

**Happy building! 🚀**


## 📝 Summary Cheat Sheet

### Quick Reference

**Creating an Agent**:
```python
agent = Agent(
    role='Role Name',
    goal='What to achieve',
    backstory='Context and personality',
    llm=llm_wrapper,  # Optional
    allow_delegation=False,  # Optional
    verbose=True  # Optional
)
```

**Creating a Task**:
```python
task = Task(
    description='What needs to be done',
    agent=assigned_agent,
    expected_output='Format of output',
    context=[other_task]  # Optional: list of tasks whose outputs will be used
)
```

**Creating a Crew**:
```python
crew = Crew(
    agents=[agent1, agent2],
    tasks=[task1, task2],
    verbose=True
)
result = crew.kickoff()
```

### Common Patterns

**Sequential Workflow**:
```
Task 1 → Task 2 (depends on Task 1) → Task 3 (depends on Task 2)
```

**Parallel + Synthesis**:
```
Task 1 ──┐
Task 2 ──┼→ Synthesis Task
Task 3 ──┘
```

**Complex Workflow**:
```
Task 1 → Task 2 ──┐
                  ├→ Task 5
Task 3 → Task 4 ──┘
```

---

**End of Notebook** 🎓
